In [32]:
import numpy as np
import torch

import sys

sys.path.append('../training/')
sys.path.append('../models/')


In [33]:
model = torch.load('../saved_models/Gatech_simple_VFb1_epoch_709.pkl',map_location='cpu')


In [34]:
sub = np.array([-8.728156899451278e-7])
div = np.sqrt(0.021395351737737656 + 1e-5)
mul = np.array([3.715294599533081])
add = np.array([0.1172400712966919])

insg_add = -sub/div*mul+add
insg_mul = mul/div

In [35]:
print(insg_add,insg_mul)


[0.11726224] [25.39405594]


In [36]:
Layer1 = [model.conv.weight.detach().numpy(), model.conv.bias.detach().numpy()]
# print(Layer1)

In [37]:
conv_weight = Layer1[0]
conv_bias = Layer1[1]

print(conv_weight.shape)
print(conv_bias.shape)

print(conv_weight[0,0,:5])
print(conv_bias)


conv_bias = conv_bias + np.sum(np.sum(insg_add.reshape(1,1,-1)*conv_weight,axis=2),axis=1)
conv_weight = conv_weight*insg_mul

print(conv_weight.shape,conv_bias.shape)
print(conv_weight[0,0,:5])
print(conv_bias)

(3, 1, 85)
(3,)
[-0.05501764  0.14627229 -0.2395012   0.23322591 -0.07594037]
[-0.27059504  0.00615436 -0.00738623]
(3, 1, 85) (3,)
[-1.397121    3.71444663 -6.0819067   5.92255185 -1.92843402]
[-0.64979663 -0.01348934 -0.01429403]


In [38]:
Layer_start = model.start.delta_in.detach().numpy()

print(Layer_start.shape)
print(Layer_start)
conv_bias_f = conv_bias/Layer_start.reshape(-1)
conv_weight_f = conv_weight/Layer_start.reshape(-1,1,1)

print(conv_weight_f.shape,conv_bias_f.shape)
print(conv_weight_f[0,0,:5])
print(conv_bias_f)

(1, 3, 1)
[[[0.0003278 ]
  [0.0033114 ]
  [0.00452419]]]
(3, 1, 85) (3,)
[ -4262.11002491  11331.43100469 -18553.69397564  18067.56007585
  -5882.953555  ]
[-1982.29410796    -4.07360858    -3.15946597]


In [39]:
Layer2 = [model.Tlayers[1].quant_weight.detach().numpy(),model.Tlayers[1].down_scaler.integer_bias.detach().reshape(-1).numpy(),model.Tlayers[1].down_scaler.right_shift.detach().reshape(-1).numpy()]

fc2_weights = Layer2[0]
fc2_bias = Layer2[1]
fc2_right_shift = -Layer2[2]

print(fc2_weights.shape, fc2_bias.shape,fc2_right_shift.shape)

(20, 111) (20,) (20,)


In [40]:
print(fc2_weights.reshape(-1)[:5])
print(fc2_bias.reshape(-1)[:5])
print(fc2_right_shift.reshape(-1)[:5])

[2. 2. 2. 2. 1.]
[7108. 4746. 4776. 4387. 3945.]
[5. 5. 5. 4. 5.]


In [41]:
fc2_weights_m = np.zeros_like(fc2_weights)
M2 = 111
for i in range(20):
    for j in range(M2//3):
        for t in range(3):
            fc2_weights_m[i][j*3+t]=fc2_weights[i][j + t *(M2 // 3)]

fc2_weights = fc2_weights_m

fc2_weights_temp = np.zeros((fc2_weights.shape[0],fc2_weights.shape[1]+1))

for i in range(0,20):
    for j in range(0,111,4):
        fc2_weights_temp[i,j] = fc2_weights[i,j]
        fc2_weights_temp[i,j+1] = fc2_weights[i,j+2]
        fc2_weights_temp[i,j+2] = fc2_weights[i,j+1]
        if j < 108:
            fc2_weights_temp[i,j+3] = fc2_weights[i,j+3]

fc2_weights = fc2_weights_temp

In [42]:
Layer3 = [model.Tlayers[2].quant_weight.detach().numpy(),model.Tlayers[2].down_scaler.integer_bias.detach().reshape(-1).numpy(),model.Tlayers[2].down_scaler.right_shift.detach().reshape(-1).numpy()]

fc3_weights = Layer3[0]
fc3_bias = Layer3[1]
fc3_right_shift = -Layer3[2]

print(fc3_weights.shape, fc3_bias.shape,fc3_right_shift.shape)

(2, 20) (2,) (2,)


In [43]:
fc3_weights_temp = np.zeros((fc3_weights.shape[0],fc3_weights.shape[1]+1))

for i in range(0,2):
    for j in range(0,20,4):
        fc3_weights_temp[i,j] = fc3_weights[i,j]
        fc3_weights_temp[i,j+1] = fc3_weights[i,j+2]
        fc3_weights_temp[i,j+2] = fc3_weights[i,j+1]
        fc3_weights_temp[i,j+3] = fc3_weights[i,j+3]

fc3_weights = fc3_weights_temp

In [44]:
print(fc3_weights.reshape(-1)[:5])
print(fc3_bias.reshape(-1)[:5])
print(fc3_right_shift.reshape(-1)[:5])

[-3. 25. 15.  5.  2.]
[-43318.  26092.]
[6. 6.]


In [45]:
Layer_stop = [model.stop.exp.detach().numpy().reshape(-1)]
float_conversion = 2**Layer_stop[0]

print(float_conversion)

[0.0003096 0.000514 ]


In [46]:
file = "./weights_Quant.hpp"


In [47]:
with open(file,mode='w+') as f:
    f.write('''#include "network_templates.hpp" 
#include <cmath>

const auto conv_ACT = [](const float &val) -> int16_t { 
    int32_t x=(int32_t)fabsf(val);
    int32_t b = (32767 - x); 
    int32_t d = (b >> 31);
    return static_cast<int16_t>((x & ~d) | (32767 & d)); 
};
            
const auto fc2_ACT = [](int32_t val, int8_t shift) -> int16_t { 
	int32_t a{val>>shift};
	a = (a & (~(a>>31)));
	int32_t b = (32767 - a); 
	int32_t d = (b >> 31);
	return static_cast<int16_t>((a & ~d) | (32767 & d));
};            
            
const auto fc3_ACT = [](int32_t val, int8_t shift) -> int32_t { return val>>shift; };

// This is bad, like really bad, who in their right mind thinks passing a void pointer is a good idea?, and cast it to a SPECIFIC TYPE which DIFFERS between the two activation functions.              
void fc2_ACT_garbage(int32_t val, int8_t shift, void* out){ 
	int32_t a{val>>shift};
	a = (a & (~(a>>31)));
	int32_t b = (32767 - a); 
	int32_t d = (b >> 31);
	*(int16_t*)out = static_cast<int16_t>((a & ~d) | (32767 & d));
}       
            
void fc3_ACT_garbage(int32_t val, int8_t shift, void* out){ 
	*(int32_t*)out = val>>shift; 
}

'''
)
    
    # CONV
    f.write('const Matrix<85,3> Conv_weight{')
    for i in range(85):
        # f.write('{')
        for q in range(3):
            f.write(f'{conv_weight_f[q,0,i]}f ')
            f.write(',')
        # f.write('}')
    f.write('};\n')

    f.write('const Matrix<3,1> Conv_bias{')
    for q in range(3):
        # f.write('{')
        f.write(f'{conv_bias_f[q]}f ')
        f.write(',')
        # f.write('}')
    f.write('};\n\n\n')


    # FC2
    f.write('// gets optimized away anyway, but ist nice to have\n')
    f.write('const Matrix<20,112,int8_t> fc2_weight{')
    for q in range(20):
        # f.write('{')
        for i in range(112):
            f.write(f'{fc2_weights[q,i].astype(int)} ')
            f.write(',')
        # f.write('}')
    f.write('};\n')


    f.write('const Matrix<20,1,int32_t> fc2_bias{')
    for q in range(20):
        # f.write('{')
        f.write(f'{fc2_bias[q].astype(int)} ')
        f.write(',')
        # f.write('}')
    f.write('};\n')

    f.write('const Matrix<20,1,int8_t> fc2_right_shift{')
    for q in range(20):
        # f.write('{')
        f.write(f'{fc2_right_shift[q].astype(int)} ')
        f.write(',')
        # f.write('}')
    f.write('};\n\n\n')
    
    # FC3
    f.write('const Matrix<2,20,int8_t> fc3_weight{')
    for q in range(2):
        # f.write('{')
        for i in range(20):
            f.write(f'{fc3_weights[q,i].astype(int)} ')
            f.write(',')
        # f.write('}')
    f.write('};\n')


    f.write('const Matrix<2,1,int32_t> fc3_bias{')
    for q in range(2):
        # f.write('{')
        f.write(f'{fc3_bias[q].astype(int)} ')
        f.write(',')
        # f.write('}')
    f.write('};\n\n\n')

    f.write('const Matrix<2,1,int8_t> fc3_right_shift{')
    for q in range(2):
        # f.write('{')
        f.write(f'{fc3_right_shift[q].astype(int)} ')
        f.write(',')
        # f.write('}')
    f.write('};\n\n\n')

    f.write(f'const float float_conversion[2]{{ {float_conversion[0]}f , {float_conversion[1]}f  }};\n\n')
    
   

In [48]:
fc2_eq=np.zeros(((20*112)//4),dtype=np.uint32)
fc2_tmp = np.uint32(fc2_weights.reshape(-1))
print(fc2_eq.shape, fc2_tmp.shape)
fc2_eq[:] = fc2_tmp[::4]<<24 | fc2_tmp[1::4]<<16 | fc2_tmp[2::4]<<8 | fc2_tmp[3::4]

np.save('./fc2_weights.npz',fc2_eq)

(560,) (2240,)


In [49]:
import plotly.express as ex

In [50]:
ex.histogram(x=[fc2_weights_m[a,:] for a in range(20)])


In [51]:
compression_char_array = []

for q in range(20):
    for i in range(112):
        compression_char_array.append(fc2_weights[q,i].item())

In [52]:
print(len(compression_char_array))


2240


In [53]:
compression_char_array_np = np.array(compression_char_array,dtype=np.int8) 


In [54]:
compression_char_array_np = np.array(compression_char_array,dtype=np.uint8) 

for number_of_bits in range(8):
    lower_half = np.right_shift(np.left_shift(compression_char_array_np,number_of_bits),number_of_bits)
    upper_half = np.right_shift(compression_char_array_np,number_of_bits) 

    value, count = np.unique(upper_half,return_counts=True)
    size= np.sum(np.sort(count)[::-1]*(np.arange(len(count))+1))
    total_size = size  + len(lower_half)*number_of_bits
    print(number_of_bits,size,total_size, 8*len(compression_char_array_np), 8*len(compression_char_array_np)-total_size)


0 71913 71913 17920 -53993
1 37956 40196 17920 -22276
2 19961 24441 17920 -6521
3 10918 17638 17920 282
4 6392 15352 17920 2568
5 4242 15442 17920 2478
6 3344 16784 17920 1136
7 3080 18760 17920 -840


In [55]:
number_of_bits = 4

lower_half = np.right_shift(np.left_shift(compression_char_array_np,number_of_bits),number_of_bits)
upper_half = np.right_shift(compression_char_array_np,number_of_bits) 

value, count = np.unique(upper_half,return_counts=True)
size= np.sum(np.sort(count)[::-1]*(np.arange(len(count))+1)) + len(lower_half)*number_of_bits



idx=np.argsort(count)[::-1]
print(value[idx], len(idx))
encoded_values = value[idx]
num_of_ones=np.arange(len(count))
dict_encoding=dict([(encoded_values[i],'1'*num_of_ones[i]+'0') for i in range(len(count))])

print(dict_encoding)
bit_stream = ''
for i in range(len(upper_half)):
    bit_stream += dict_encoding[upper_half[i]]

print(bit_stream)
print(len(bit_stream))



[ 0 15 14  1 13  2 12  3  4 11 10  5  6  8  9  7] 16
{0: '0', 15: '10', 14: '110', 1: '1110', 13: '11110', 2: '111110', 12: '1111110', 3: '11111110', 4: '111111110', 11: '1111111110', 10: '11111111110', 5: '111111111110', 6: '1111111111110', 8: '11111111111110', 9: '111111111111110', 7: '1111111111111110'}
00100000100100110010100100101010100100101000000111010010011010010010001001001101000010111001110011100010010010101110100001101100110011111111101101011111001111100111110111001111001111011111101111111111011110011111100111110111110011111110011111111111010011111100111111111001111110111100000010010000000101001001101010100110010000010100000000010010101000000100100101101010000111000100110110101000010001000001001001010011110011111101101101111111001111111111100111111110111111100110011011111111101111111111011110011111111100000000000010010011001101001001010101001100000001110110101111001111110011011001111100111111101111111011111111000110011111111101111001111001111001110000010100000111011101001001

In [56]:
bit_stream_padded = bit_stream + '0'*(32-len(bit_stream)%32)
print(bit_stream)
print(bit_stream_padded)
print(bit_stream_padded)

0010000010010011001010010010101010010010100000011101001001101001001000100100110100001011100111001110001001001010111010000110110011001111111110110101111100111110011111011100111100111101111110111111111101111001111110011111011111001111111001111111111101001111110011111111100111111011110000001001000000010100100110101010011001000001010000000001001010100000010010010110101000011100010011011010100001000100000100100101001111001111110110110111111100111111111110011111111011111110011001101111111110111111111101111001111111110000000000001001001100110100100101010100110000000111011010111100111111001101100111110011111110111111101111111100011001111111110111100111100111100111000001010000011101110100100110110011001000100001000111000001001001011001001001010010010100011100010001010000000011001101010000011101100000101010010010100000101101001001000000101000000111001100110111111111011111111101001100111111101111100111111100111111110011011110011111111100111111011001001110100100100010011100111111101110111111101001

In [57]:
def reconstruct_data(lower_half, bit_stream, dict_decoding:dict):
    out = np.zeros((20,112),dtype=np.int8)
    p=0
    for q in range(20):
        for i in range(112):
            out[q,i] = lower_half[p]
            p+=1

    field = ''
    q=0
    i=0
    for bit in bit_stream:
        field += bit
        if bit=='0':
            value = len(field)-1
            value = dict_decoding[value]
            out[q,i] = np.bitwise_or(out[q,i],np.uint8(value)<<4)
            field = ''
            i+=1
            if i==112:
                i=0
                q+=1
            if q==20:
                break

    return out


In [58]:
keys, values = dict_encoding.keys() , dict_encoding.values()
values = [len(a)-1 for a in values]
dict_decoding = dict(zip(values,keys))

out = reconstruct_data(lower_half,bit_stream,dict_decoding)


In [59]:
print(dict_decoding)
decode_array = np.left_shift(np.array(list(dict_decoding.values())),4)
print(decode_array)
print(','.join([str(a) for a in decode_array]))

{0: 0, 1: 15, 2: 14, 3: 1, 4: 13, 5: 2, 6: 12, 7: 3, 8: 4, 9: 11, 10: 10, 11: 5, 12: 6, 13: 8, 14: 9, 15: 7}
[  0 240 224  16 208  32 192  48  64 176 160  80  96 128 144 112]
0,240,224,16,208,32,192,48,64,176,160,80,96,128,144,112


In [60]:
print(out-fc2_weights)
print(np.all((out-fc2_weights)==0))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
True


In [61]:
bit_stream_padded_np = np.array([int(bit_stream_padded[i:i+32], 2) for i in range(0, len(bit_stream_padded), 32)], dtype=np.uint32)


In [62]:


with open(file,mode='a+') as f:
    # FC2 Compressed
    f.write('const uint32_t upper_half[]{')
    for a in range(bit_stream_padded_np.shape[0]):
        f.write(f'{bit_stream_padded_np[a]},')
    f.write('};')
    f.write('\n\n')

    f.write('const uint8_t lower_half[]{')
    for i in range(len(lower_half)//2):
            f.write(f'{lower_half[i*2].astype(np.uint8)<<4 | lower_half[i*2+1].astype(np.uint8)} ')
            f.write(',')
        # f.write('}')
    f.write('};\n')

    f.write(f'const uint8_t decoder[]{{{", ".join([str(a) for a in decode_array])}}};')
